In [1]:
import pandas as pd
import sklearn.datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sasctl import Session
from sasctl.tasks import register_model, publish_model, update_model_performance
from sasctl.services import model_repository as mr
from sasctl.services import model_management as mm

In [2]:
data = sklearn.datasets.load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame(data.target, columns=['Price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
# Establish a session with SAS Viya
Session('http://frasepviya35smp', 'viyademo01', 'demopw')

project = 'Boston Housing'
model_name = 'Boston Regression'

In [5]:
# Fit a linear regression model using sci-kit learn
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [4]:
# Register the model in SAS Model Manager
register_model(lm,
               model_name,
               input=X_train,       # Use X to determine model inputs
               project=project,     # Register in "Boston Housing" project
               force=True)          # Create project if it doesn't exist

HTTPError: HTTP Error 409: {"errorCode":0,"message":"The model \"Boston Regression\" already exists in the specified location.","details":["traceId: 85bc6109a25ece93","path: /modelRepository/models"],"links":[],"version":2,"httpStatusCode":409}

In [9]:
# Update project properties.  Target variable must be set before performance
# definitions can be created.
project = mr.get_project(project)
project['targetVariable'] = 'Price'
project = mr.update_project(project)

In [11]:
# Publish the model to the real-time scoring engine
module_lm = publish_model(model_name, 'maslocal')

In [16]:
# Select the first row of testing data
x = X_test.iloc[0, :]
x

CRIM         0.04417
ZN          70.00000
INDUS        2.24000
CHAS         0.00000
NOX          0.40000
RM           6.87100
AGE         47.40000
DIS          7.82780
RAD          5.00000
TAX        358.00000
PTRATIO     14.80000
B          390.86000
LSTAT        6.07000
Name: 300, dtype: float64

In [17]:
# Call the published module and score the record
result = module_lm.predict(x)
print(result)

None


In [18]:
# Build a second model
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [19]:
# Register the second model in Model Manager
model_dt = register_model(dt, 'Decision Tree', project, input=X)

In [21]:
# Publish from Model Manager -> MAS
module_dt = publish_model(model_dt, 'maslocal')

In [22]:
# Use MAS to score some new data
result = module_dt.predict(x)
print(result)

None


In [24]:
# Instruct the project to look for tables in the "Public" CAS library with
# names starting with "boston_" and use these tables to track model
# performance over time.
mm.create_performance_definition(model_name, 'Public', 'boston')

# Model Manager can track model performance over time if provided with
# historical model observations & predictions.  SIMULATE historical data by
# repeatedly sampling from the test set.
perf_df = X_test.copy()
perf_df['var1'] = lm.predict(X_test)
perf_df['Price'] = y

# For each (simulated) historical period, upload model results
for period in ('q1', 'q2', 'q3', 'q4'):
    sample = perf_df.sample(frac=0.2)
    update_model_performance(sample, model_name, period)

HTTPError: HTTP Error 409: {"errorCode":21307,"message":"The performance task is already running.","details":["traceId: 1b653d97d9682bbb","path: /modelManagement/performanceTasks/573a5799-c7e5-4e66-bcd1-607f585b5f65"],"links":[],"version":2,"httpStatusCode":409}